In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [36]:
# Data Loading, Preparing and Scaling
df = pd.read_csv("./dataset.csv")
input_n = df.drop(['Production','Year'], axis='columns')
input_n

,Province,Harvested Area,Rainfall,Humidity,Temperature
0,Aceh,329516,2336,81,28
1,Aceh,310012,1437,82,27
2,Aceh,317869,1790,76,29
3,Aceh,297058,2293,76,29
4,Aceh,271750,1834,76,29
...,...,...,...,...,...
199,Papua,54132,1823,77,28
200,Papua,52728,1502,75,28
201,Papua,64985,2028,76,28
202,Papua,49742,2576,84,28


In [37]:
target = df['Production']
target

0      1861567
1      1714438
2      1757313
3      1634640
4      1509456
        ...   
199     235340
200     166002
201     286280
202     193944
203     200115
Name: Production, Length: 204, dtype: int64

In [38]:
from sklearn.preprocessing import LabelEncoder
le_province = LabelEncoder()
input_n['Province'] = le_province.fit_transform(input_n['Province'])
input_n

,Province,Harvested Area,Rainfall,Humidity,Temperature
0,0,329516,2336,81,28
1,0,310012,1437,82,27
2,0,317869,1790,76,29
3,0,297058,2293,76,29
4,0,271750,1834,76,29
...,...,...,...,...,...
199,23,54132,1823,77,28
200,23,52728,1502,75,28
201,23,64985,2028,76,28
202,23,49742,2576,84,28


In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    input_n,  # your feature dataframe
    target,    # your target array/series
    test_size=0.2,      # 20% for testing
    random_state=42
)

#scaling feature
from sklearn.preprocessing import MinMaxScaler
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

#scale features
X_train_scaled = feature_scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=["Province","Harvested Area","Rainfall","Humidity","Temperature"], index=X_train.index)

# Scale target variable
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
print(X_train_scaled)
print(y_train_scaled)

#Scale test data
X_test_scaled = feature_scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=["Province","Harvested Area","Rainfall","Humidity","Temperature"], index=X_test.index)
y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1)).flatten()


     Province  Harvested Area  Rainfall  Humidity  Temperature
199  0.696970        0.029636  0.276728  0.363636          0.6
93   0.060606        0.174608  0.516504  0.272727          0.8
38   0.090909        0.035128  0.785344  0.363636          0.8
24   0.212121        0.047240  0.446959  0.727273          0.4
96   0.030303        0.060839  0.222960  0.500000          0.6
..        ...             ...       ...       ...          ...
106  0.636364        0.148176  0.448412  0.409091          0.8
14   0.939394        0.162212  0.880424  1.000000          0.6
92   0.060606        0.178497  0.458377  0.227273          1.0
179  0.787879        0.032372  0.428690  0.500000          0.6
102  0.636364        0.158688  0.244551  0.227273          0.8

[163 rows x 5 columns]
[2.23748270e-02 1.52662045e-01 2.78508815e-02 3.64431838e-02
 6.34951446e-02 6.96679212e-02 2.61100764e-02 2.19494598e-02
 4.26319617e-04 2.85469368e-02 2.31739381e-02 1.60713733e-01
 1.32682075e-01 4.94595523e-02 4.2518

In [40]:
X_train_reshaped = X_train_scaled.values.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.values.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Update your model's input_shape
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(1, 5)))  # (timesteps, features)
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train with reshaped data
history = model.fit(X_train_reshaped, y_train_scaled, epochs=100, batch_size=32, validation_split=0.1)

Epoch 1/100


/home/ashwin/Desktop/Internship/code/Indonesia-rice-prediction/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0507 - val_loss: 0.1249
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0562 - val_loss: 0.1079
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0345 - val_loss: 0.1006
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0319 - val_loss: 0.0960
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0348 - val_loss: 0.0919
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0411 - val_loss: 0.0866
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0320 - val_loss: 0.0809
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0326 - val_loss: 0.0737
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0280 - val_loss: 0.0638
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0248 - val_loss: 0.0530
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0229 - val_loss: 0.0417
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0180 - val_loss: 0.0322
E

In [41]:
predictions = model.predict(X_test_reshaped)
predictions = target_scaler.inverse_transform(predictions).flatten()
y_test = target_scaler.inverse_transform(y_test_scaled.reshape(-1,1)).flatten()

rmse = np.sqrt(np.mean((y_test - predictions)**2))
print(f'RMSE: {rmse:.2f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
RMSE: 1330017.46


In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred_scaled = model.predict(X_test_reshaped)
print(y_pred_scaled[:,0].shape)
print(y_test_scaled.shape)

# Regression metrics
mse = mean_squared_error(y_test_scaled, y_pred_scaled[:,0])
mae = mean_absolute_error(y_test_scaled, y_pred_scaled[:,0])
r2 = r2_score(y_test_scaled, y_pred_scaled[:,0])

print(f"Mean Squared Error: {mse:.6f}")
print(f"Mean Absolute Error: {mae:.6f}")
print(f"R² Score: {r2:.6f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
(41,)
(41,)
Mean Squared Error: 0.016047
Mean Absolute Error: 0.070463
R² Score: 0.851319


# Time Series Approach Using Year Information


In [43]:
# 1st cell: Load data and create input features for time series
df_ts = pd.read_csv("./dataset.csv")
print("Dataset shape:", df_ts.shape)
print("Years available:", sorted(df_ts['Year'].unique()))
print("Provinces:", df_ts['Province'].unique())

# Encode provinces for time series
from sklearn.preprocessing import LabelEncoder
le_province_ts = LabelEncoder()
df_ts['Province_encoded'] = le_province_ts.fit_transform(df_ts['Province'])

# Create input features (excluding Production which is our target)
input_n_ts = df_ts[['Province_encoded', 'Year', 'Harvested Area', 'Rainfall', 'Humidity', 'Temperature']]
print("\nInput features shape:", input_n_ts.shape)
print("\nInput features:")
print(input_n_ts.head(10))

Dataset shape: (204, 7)
Years available: [np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
Provinces: ['Aceh' 'Sumatera Utara' 'Sumatera Barat' 'Riau' 'Jambi'
 'Sumatera Selatan' 'Bengkulu' 'Lampung' 'Kepulauan Bangka Belitung'
 'Kepulauan Riau' 'DKI Jakarta' 'Jawa Barat' 'Jawa Tengah' 'DI Yogyakarta'
 'Jawa Timur' 'Banten' 'Bali' 'Nusa Tenggara Barat' 'Nusa Tenggara Timur'
 'Kalimantan Barat' 'Kalimantan Tengah' 'Kalimantan Selatan'
 'Kalimantan Timur' 'Kalimantan Utara' 'Sulawesi Utara' 'Sulawesi Tengah'
 'Sulawesi Selatan' 'Sulawesi Tenggara' 'Gorontalo' 'Sulawesi Barat'
 'Maluku' 'Maluku Utara' 'Papua Barat' 'Papua']

Input features shape: (204, 6)

Input features:
   Province_encoded  Year  Harvested Area  Rainfall  Humidity  Temperature
0                 0  2018          329516      2336        81           28
1                 0  2019          310012      1437        82           27
2                 0  2020          317869      179

In [44]:
target_ts = df_ts['Production']
print("Target variable shape:", target_ts.shape)
print("\nTarget variable (Production):")
print(target_ts.head(10))

Target variable shape: (204,)

Target variable (Production):
0    1861567
1    1714438
2    1757313
3    1634640
4    1509456
5    1393474
6    2108285
7    2078902
8    2040500
9    2004143
Name: Production, dtype: int64


In [63]:
# 3rd cell: Structure data for time series, split and scale
def create_time_series_sequences(data, target, sequence_length):
    """
    Create time series sequences for LSTM
    sequence_length: number of years to look back
    """
    sequences = []
    targets = []
    
    # Group by province to create sequences within each province
    provinces = data['Province_encoded'].unique()
    
    for province in provinces:
        # Get data for this province, sorted by year
        province_data = data[data['Province_encoded'] == province].sort_values('Year')
        province_target = target[data['Province_encoded'] == province].reindex(province_data.index)
        
        # Create sequences for this province
        for i in range(len(province_data) - sequence_length + 1):
            # Get sequence of features (excluding Year as it's just for ordering)
            seq_features = province_data.iloc[i:i+sequence_length][['Province_encoded', 'Harvested Area', 'Rainfall', 'Humidity', 'Temperature']].values
            seq_target = province_target.iloc[i+sequence_length-1]  # Predict the last year in sequence
            
            sequences.append(seq_features)
            targets.append(seq_target)
    
    return np.array(sequences), np.array(targets)

# Create time series sequences (using 3 years to predict current year)
X_sequences, y_sequences = create_time_series_sequences(input_n_ts, target_ts, 4)
print(f"Sequences shape: {X_sequences}")  # (samples, timesteps, features)
print(f"Targets shape: {y_sequences.shape}")

# Split the data
from sklearn.model_selection import train_test_split
X_train_ts, X_test_ts, y_train_ts, y_test_ts = train_test_split(
    X_sequences,
    y_sequences,
    test_size=0.2,
    random_state=42
)

# Scale features and target using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
feature_scaler_ts = MinMaxScaler(feature_range=(0, 1))
target_scaler_ts = MinMaxScaler(feature_range=(0, 1))

# Reshape for scaling (combine all timesteps and samples for consistent scaling)
X_train_reshaped_for_scaling = X_train_ts.reshape(-1, X_train_ts.shape[-1])

# print(X_train_reshaped_for_scaling.shape)
X_train_scaled_ts = feature_scaler_ts.fit_transform(X_train_reshaped_for_scaling)
X_train_scaled_ts = X_train_scaled_ts.reshape(X_train_ts.shape)
# Scale target
y_train_scaled_ts = target_scaler_ts.fit_transform(y_train_ts.reshape(-1, 1)).flatten()

# Scale test data
X_test_reshaped_for_scaling = X_test_ts.reshape(-1, X_test_ts.shape[-1])
X_test_scaled_ts = feature_scaler_ts.transform(X_test_reshaped_for_scaling)
X_test_scaled_ts = X_test_scaled_ts.reshape(X_test_ts.shape)
y_test_scaled_ts = target_scaler_ts.transform(y_test_ts.reshape(-1, 1)).flatten()

print(f"\nTraining data shape: {X_train_scaled_ts.shape}")
print(f"Training target shape: {y_train_scaled_ts.shape}")
print(f"Test data shape: {X_test_scaled_ts.shape}")
print(f"Test target shape: {y_test_scaled_ts.shape}")

Sequences shape: [[[     0 329516   2336     81     28]
  [     0 310012   1437     82     27]
  [     0 317869   1790     76     29]
  [     0 297058   2293     76     29]]

 [[     0 310012   1437     82     27]
  [     0 317869   1790     76     29]
  [     0 297058   2293     76     29]
  [     0 271750   1834     76     29]]

 [[     0 317869   1790     76     29]
  [     0 297058   2293     76     29]
  [     0 271750   1834     76     29]
  [     0 254319   2555     80     28]]

 ...

 [[    23  52412   2991     83     28]
  [    23  54132   1823     77     28]
  [    23  52728   1502     75     28]
  [    23  64985   2028     76     28]]

 [[    23  54132   1823     77     28]
  [    23  52728   1502     75     28]
  [    23  64985   2028     76     28]
  [    23  49742   2576     84     28]]

 [[    23  52728   1502     75     28]
  [    23  64985   2028     76     28]
  [    23  49742   2576     84     28]
  [    23  49323   2555     80     28]]]
Targets shape: (102,)

Traini

In [ ]:
# Data is already in correct shape for LSTM: (samples, timesteps, features)
print(f"X_train_scaled_ts shape: {X_train_scaled_ts.shape}")
print(f"X_test_scaled_ts shape: {X_test_scaled_ts.shape}")

# Build Time Series LSTM Model
model_ts = Sequential()
model_ts.add(LSTM(units=128, return_sequences=True, input_shape=(X_train_scaled_ts.shape[1], X_train_scaled_ts.shape[2])))
model_ts.add(Dropout(0.2))
model_ts.add(LSTM(units=64, return_sequences=False))
model_ts.add(Dropout(0.2))
model_ts.add(Dense(32, activation='relu'))
model_ts.add(Dense(1))

model_ts.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

print("Model architecture:")
model_ts.summary()

# Train the model
print("\nTraining the model...")
history_ts = model_ts.fit(
    X_train_scaled_ts, 
    y_train_scaled_ts, 
    epochs=100, 
    batch_size=32, 
    validation_split=0.1,
    verbose=1
)

X_train_scaled_ts shape: (81, 4, 5)
X_test_scaled_ts shape: (21, 4, 5)
Model architecture:


/home/ashwin/Desktop/Internship/code/Indonesia-rice-prediction/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_76 (LSTM)                  │ (None, 4, 128)         │        68,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_76 (Dropout)            │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_77 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_77 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_75 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 120,129 (469.25 KB)

 Trainable params: 120,129 (469.25 KB)

 Non-trainable params: 0 (0.00 B)


Training the model...
Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 113ms/step - loss: 0.1168 - mae: 0.1661 - val_loss: 0.0053 - val_mae: 0.0704
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0842 - mae: 0.1581 - val_loss: 0.0198 - val_mae: 0.1330
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0652 - mae: 0.1757 - val_loss: 0.0230 - val_mae: 0.1449
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0626 - mae: 0.1796 - val_loss: 0.0137 - val_mae: 0.1099
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0538 - mae: 0.1543 - val_loss: 0.0064 - val_mae: 0.0762
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0403 - mae: 0.1217 - val_loss: 0.0038 - val_mae: 0.0596
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0311 - mae: 0.1021 - val_loss: 0.0054 - val_mae: 0.0684
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0248 - mae: 0.1023 - val_loss: 0.0042 - val_mae: 0.0582
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/ste

In [52]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Make predictions on test data
predictions_ts = model_ts.predict(X_test_scaled_ts)

# Calculate metrics on scaled data
mse_scaled = mean_squared_error(y_test_scaled_ts, predictions_ts.flatten())
mae_scaled = mean_absolute_error(y_test_scaled_ts, predictions_ts.flatten())
r2_scaled = r2_score(y_test_scaled_ts, predictions_ts.flatten())


print("=== Time Series LSTM Results ===")
print("\nScaled Data Metrics:")
print(f"Mean Squared Error: {mse_scaled:.6f}")
print(f"Mean Absolute Error: {mae_scaled:.6f}")
print(f"R² Score: {r2_scaled:.6f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
=== Time Series LSTM Results ===

Scaled Data Metrics:
Mean Squared Error: 0.004052
Mean Absolute Error: 0.043438
R² Score: 0.947912


In [53]:
# 6th cell: Hyperparameter tuning for maximum accuracy
from tensorflow.keras.callbacks import EarlyStopping
import itertools

print("=== Hyperparameter Tuning ===")

# Define hyperparameter combinations to try
lstm_units_options = [64, 128, 256]
dropout_rates = [0.1, 0.2, 0.3]
batch_sizes = [16, 32, 64]
learning_rates = [0.001, 0.01]

best_r2 = -float('inf')
best_params = {}
best_model = None

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Grid search through hyperparameters (simplified for computational efficiency)
param_combinations = [
    {'lstm_units': 128, 'dropout': 0.2, 'batch_size': 32, 'lr': 0.001},
    {'lstm_units': 256, 'dropout': 0.1, 'batch_size': 32, 'lr': 0.001},
    {'lstm_units': 64, 'dropout': 0.3, 'batch_size': 16, 'lr': 0.01},
    {'lstm_units': 128, 'dropout': 0.1, 'batch_size': 64, 'lr': 0.001},
    {'lstm_units': 256, 'dropout': 0.2, 'batch_size': 16, 'lr': 0.01}
]

for i, params in enumerate(param_combinations):
    print(f"\nTesting combination {i+1}/{len(param_combinations)}: {params}")
    
    # Build model with current hyperparameters
    model_tuned = Sequential()
    model_tuned.add(LSTM(units=params['lstm_units'], return_sequences=True, 
                        input_shape=(X_train_scaled_ts.shape[1], X_train_scaled_ts.shape[2])))
    model_tuned.add(Dropout(params['dropout']))
    model_tuned.add(LSTM(units=params['lstm_units']//2, return_sequences=False))
    model_tuned.add(Dropout(params['dropout']))
    model_tuned.add(Dense(32, activation='relu'))
    model_tuned.add(Dense(1))
    
    # Compile with current learning rate
    from tensorflow.keras.optimizers import Adam
    optimizer = Adam(learning_rate=params['lr'])
    model_tuned.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    
    # Train model
    history_tuned = model_tuned.fit(
        X_train_scaled_ts, 
        y_train_scaled_ts,
        epochs=50,  # Reduced for tuning efficiency
        batch_size=params['batch_size'],
        validation_split=0.1,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Evaluate model
    predictions_tuned = model_tuned.predict(X_test_scaled_ts, verbose=0)
    r2_tuned = r2_score(y_test_scaled_ts, predictions_tuned.flatten())
    
    print(f"R² Score: {r2_tuned:.6f}")
    
    # Update best model if this one is better
    if r2_tuned > best_r2:
        best_r2 = r2_tuned
        best_params = params.copy()
        best_model = model_tuned
        
print(f"\n=== Best Hyperparameters Found ===")
print(f"Best parameters: {best_params}")
print(f"Best R² Score: {best_r2:.6f}")

# Final evaluation with best model
print(f"\n=== Final Tuned Model Results ===")
best_predictions = best_model.predict(X_test_scaled_ts)

# Metrics on scaled data
best_mse_scaled = mean_squared_error(y_test_scaled_ts, best_predictions.flatten())
best_mae_scaled = mean_absolute_error(y_test_scaled_ts, best_predictions.flatten())
best_r2_scaled = r2_score(y_test_scaled_ts, best_predictions.flatten())

print("\nBest Model - Scaled Data Metrics:")
print(f"Mean Squared Error: {best_mse_scaled:.6f}")
print(f"Mean Absolute Error: {best_mae_scaled:.6f}")
print(f"R² Score: {best_r2_scaled:.6f}")


=== Hyperparameter Tuning ===

Testing combination 1/5: {'lstm_units': 128, 'dropout': 0.2, 'batch_size': 32, 'lr': 0.001}


/home/ashwin/Desktop/Internship/code/Indonesia-rice-prediction/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


R² Score: -0.100172

Testing combination 2/5: {'lstm_units': 256, 'dropout': 0.1, 'batch_size': 32, 'lr': 0.001}


/home/ashwin/Desktop/Internship/code/Indonesia-rice-prediction/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


R² Score: 0.903468

Testing combination 3/5: {'lstm_units': 64, 'dropout': 0.3, 'batch_size': 16, 'lr': 0.01}


...

R² Score: 0.569518

Testing combination 4/5: {'lstm_units': 128, 'dropout': 0.1, 'batch_size': 64, 'lr': 0.001}


/home/ashwin/Desktop/Internship/code/Indonesia-rice-prediction/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


R² Score: -0.090683

Testing combination 5/5: {'lstm_units': 256, 'dropout': 0.2, 'batch_size': 16, 'lr': 0.01}


/home/ashwin/Desktop/Internship/code/Indonesia-rice-prediction/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


R² Score: 0.912559

=== Best Hyperparameters Found ===
Best parameters: {'lstm_units': 256, 'dropout': 0.2, 'batch_size': 16, 'lr': 0.01}
Best R² Score: 0.912559

=== Final Tuned Model Results ===
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

Best Model - Scaled Data Metrics:
Mean Squared Error: 0.006802
Mean Absolute Error: 0.046638
R² Score: 0.912559
